In [1]:
from data_loader import load_acc_data, load_params, load_vol_data
from classifiers import get_default_classifiers, set_classifier, get_online_predictions
from utilities import get_all_windows, extract_windows_data, apply_volume_filter, rename_columns, classify_by_access, evaluate_model

In [2]:
params = load_params()
df = load_acc_data("Pop2")
df_vol = load_vol_data("Pop2")
print(params)
df.head()

{'window_size': 1, 'step_size': 1, 'model': 'GP'}


,NameSpace,27_2013,28_2013,29_2013,30_2013,31_2013,32_2013,33_2013,34_2013,35_2013,...,12_2014,13_2014,14_2014,15_2014,16_2014,17_2014,18_2014,19_2014,20_2014,21_2014
0,0,2,21,152,20,25,0,1,0,29,...,0,1,9,6,0,0,3,7,13,10
1,1,15,1,0,0,0,0,0,2,6,...,0,0,0,0,0,0,0,0,0,0
2,2,955,2241,3801,2361,39,0,311,1432,3568,...,2356,2885,3322,3628,2001,2120,5981,4135,3621,1843
3,3,399,82,106,215,5,0,3,22,11,...,5,41,10,36,10,0,12,28,0,0
4,4,15,9,6,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
params['model'] = 'GP'

In [5]:
# exec.py

params = load_params()
params['model'] = 'GP'  # Defina o modelo como Programação Genética

if params['model'] != "ONL":
    clfs_dict = get_default_classifiers()
    clf = set_classifier(params['model'], clfs_dict)

    number_of_weeks = len(df.columns) - 1  # Ignorando a coluna de namespace
    last_week_of_training = number_of_weeks - params['window_size'] * 2 

    for first_week in range(0, last_week_of_training, params['step_size']):
        windows = get_all_windows(first_week, params['window_size'], params['step_size'])
        last_week_of_training = windows['x_train'][1]
        
        # Extrair os dados das janelas
        data_dict = extract_windows_data(df, df_vol, windows)
        
        # Aplicar o filtro de volume nos dados
        filtered_data = apply_volume_filter(data_dict, df_vol, last_week_of_training)
        
        # Renomear as colunas para serem consistentes
        renamed_data = rename_columns(filtered_data)
        
        # Transformar os dados de rótulo em binário
        renamed_data['y_train'] = classify_by_access(renamed_data['y_train'])
        renamed_data['y_test'] = classify_by_access(renamed_data['y_test'])
        
        # Treinar o modelo
        clf.fit(renamed_data['x_train'], renamed_data['y_train'])
        
        # Testar o modelo
        y_pred = clf.predict(renamed_data['x_test'])
        y_prob = clf.predict_proba(renamed_data['x_test'])[:, 1]  # Probabilidade da classe positiva
        
        # Avaliar o modelo com a métrica de acurácia
        metrics = evaluate_model(renamed_data['y_test'].ravel(), y_pred, y_prob)
        
        # Imprimir as métricas
        print(metrics)
else:
    # Código existente para o modo "ONL"
    pass


ONL
{'accuracy': 0.8277274056741737, 'precision': 0.7084717607973422, 'recall': 0.7818515123739689, 'f1_score': 0.7433551198257081, 'auc_roc': 0.81553915824884, 'true_positive': 853, 'true_negative': 1977, 'false_positive': 351, 'false_negative': 238}
